#Stock Indicators Python

In [15]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [16]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279190,85.324337,83.909667,84.556805,118944600
1,2010-01-05,85.504951,85.542578,84.918014,85.226535,111579900
2,2010-01-06,85.565140,85.775835,85.354445,85.422166,116074400
3,2010-01-07,85.926353,86.031701,85.166341,85.407136,131091100
4,2010-01-08,86.212280,86.249907,85.527521,85.700590,126402800


In [17]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [ ]:
# Moving Average Convergence / Divergence (MACD)
df['macd'] = [r.macd for r in indicators.get_macd(quotes, 12, 26, 9)]
df['signal'] = [r.signal for r in indicators.get_macd(quotes, 12, 26, 9)]
df['histogram'] = [r.histogram for r in indicators.get_macd(quotes, 12, 26, 9)]
df['bullishMACD'] = 0.0
df['bullishMACD'] = np.where(df['macd'] > df['signal'], 1.0, 0.0)
df['crossover_MACD'] = df['bullishMACD'].diff()

# Williams Alligator
df['jaw'] = [r.jaw for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]
df['teeth'] = [r.teeth for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]
df['lips'] = [r.lips for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]

# Calculate EMA
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['EMA 20'] = [r.ema for r in indicators.get_ema(quotes, 20)]
df['EMA 25'] = [r.ema for r in indicators.get_ema(quotes, 25)]

# Calculate EMA12 crossover EMA25
df['bullishEMA'] = 0.0
df['bullishEMA'] = np.where(df['EMA 12'] > df['EMA 25'], 1.0, 0.0)
df['crossover_EMA12_EMA25'] = df['bullishEMA'].diff()

# Calculate Chandelier exit
from stock_indicators import ChandelierType 
df['chandelier_long_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.LONG)]
df['chandelier_short_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.SHORT)]

# Calculate Bollinger Bands
df['BB_upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]




# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima12'] = df['rsi'].rolling(12).mean()

# Calculate Mcginley dynamic
df['dynamic20'] = [r.dynamic for r in indicators.get_dynamic(quotes, 20)]


In [19]:
df = df.dropna().reset_index(drop=True)
df.head()


,Date,Close,High,Low,Open,Volume,macd,signal,histogram,bullishMACD,...,BB_upper_band,BB_middle_band,BB_lower_band,donchian_upper,donchian_lower,donchian_center,rsi,rsima6,rsima12,dynamic20
0,2010-02-22,83.646332,83.962374,83.398010,83.939801,132346900,-0.676999,-1.432504,0.755505,1.0,...,84.319313,81.898289,79.477265,84.08274280612169,78.69496011788378,81.388851462002735,55.167589,51.564535,44.354506,82.522024
1,2010-02-23,82.630447,83.676402,82.412226,83.420560,207497000,-0.617653,-1.269534,0.651881,1.0,...,84.322599,81.899794,79.476989,83.96237447361706,78.69496011788378,81.32866729575042,49.034543,52.562951,45.610669,82.531012
2,2010-02-24,83.390465,83.525912,82.668079,82.878774,176350700,-0.503490,-1.116325,0.612835,1.0,...,84.461876,81.956607,79.451339,83.96237447361706,78.69496011788378,81.32866729575042,53.224190,53.107538,47.134856,82.599726
3,2010-02-25,83.277580,83.337780,81.975783,82.201526,259634700,-0.417314,-0.976523,0.559209,1.0,...,84.542904,81.988211,79.433518,83.96237447361706,78.69496011788378,81.32866729575042,52.533421,53.216368,48.754683,82.654397
4,2010-02-26,83.330276,83.616225,82.856212,83.352850,173589300,-0.340837,-0.849386,0.508549,1.0,...,84.685788,82.069855,79.453923,83.96237447361706,78.69496011788378,81.32866729575042,52.841114,52.981854,49.903031,82.708915


In [ ]:
# Moving Average Convergence / Divergence (MACD)
if __name__ == '__main__':
    
    chart = Chart(title="MACD Crossover", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    macd_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    macd_line = macd_chart.create_line('macd', color="#2ee30f", width=1, price_label=False)
    macd_line.set(df[['Date', 'macd']])

    signal_line = macd_chart.create_line('signal', color="#f36021", width=1, price_label=False)
    signal_line.set(df[['Date', 'signal']])

    histogram_bar = macd_chart.create_histogram('histogram', color="#1341e4", price_label=False)
    histogram_bar.set(df[['Date', 'histogram']])


    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        MACD_diff = df.iloc[i]['crossover_MACD']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if MACD_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif MACD_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [ ]:
# Williams Alligator
if __name__ == '__main__':
    
    chart = Chart(title="Williams Alligator", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    jaw_line = chart.create_line('jaw', color="#1f77b4", width=1, price_label=False)
    jaw_line.set(df[['Date', 'jaw']])
    teeth_line = chart.create_line('teeth', color="#ff7f0e", width=1, price_label=False)
    teeth_line.set(df[['Date', 'teeth']])
    lips_line = chart.create_line('lips', color="#2ca02c", width=1, price_label=False)
    lips_line.set(df[['Date', 'lips']])

chart.show(block = True)

In [ ]:
# Bollinger Bands
if __name__ == '__main__':

    chart = Chart(title="Bollinger Bands", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    upper_band_line = chart.create_line('BB_upper_band', color="#ff0000", width=1, price_label=False)
    upper_band_line.set(df[['Date', 'BB_upper_band']])
    middle_band_line = chart.create_line('BB_middle_band', color="#f3bd0b", width=1, price_label=False)
    middle_band_line.set(df[['Date', 'BB_middle_band']])
    lower_band_line = chart.create_line('BB_lower_band', color="#00ff00", width=1, price_label=False)
    lower_band_line.set(df[['Date', 'BB_lower_band']])

chart.show(block = True)

In [ ]:
# Now, let's create the chart with lightweight-charts and add the EMAs and buy/sell markers.

# Assuming `df` is already a pandas DataFrame with 'Date', 'Open', 'High', 'Low', 'Close', 'EMA 12', and 'EMA 25' columns.
# It's good practice to convert the 'Date' column to the correct datetime format.
# df['Date'] = pd.to_datetime(df['Date'])

if __name__ == '__main__':
    
    chart = Chart(title="EMA12_EMA25 Crossover", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=False)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_label=False)
    ema25_line.set(df[['Date', 'EMA 25']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        emadiff = df.iloc[i]['crossover_EMA12_EMA25']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if emadiff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif emadiff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)
